In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# a)

trend = lambda x : 5 * x ** 2 + 3 * x + 1
sezon = lambda x: np.sin(2 * np.pi * 5 * x) + np.sin(2 * np.pi * 7 * x)
variatii = lambda x:  np.random.normal(0,1,len(x))

N = 1000
time = np.linspace(0, 1, N)
t = trend(time)
s = sezon(time)
v = variatii(time)

signal = t + s + v 

fig, axs = plt.subplots(4)
axs[0].plot(time, t)
axs[0].set_title("trend")
axs[1].plot(time, s)
axs[1].set_title("sezon")
axs[2].plot(time, v)
axs[2].set_title("variatii")
axs[3].plot(time, signal)
axs[3].set_title("semnal")

fig.savefig("ex1_a.pdf")
fig.show()



In [ ]:
# b)

signalAltered = (signal - np.mean(signal)) / (np.std(signal))
correlation_vector = np.correlate(signalAltered, signalAltered, mode="full") 
correlation_vector = correlation_vector[correlation_vector.size // 2:] / N

plt.plot(correlation_vector)
plt.savefig("ex2_b.pdf")
plt.show()

In [58]:
# c)

p = 2

def train(trainSignal, p):
    Y =  trainSignal[trainSignal.size - 2 : 0 : -1]
    print(Y.shape)
    for i in range(2, p + 1):
        shifted = trainSignal[trainSignal.size - p - 1 : i - 1 : -1]
        Y = np.column_stack((Y, shifted))

    print(trainSignal.shape)
    x_star = np.linalg.lstsq(Y, trainSignal, rcond=None)[0]
    
    return x_star

foldSize = 800
trainSignal = signal[: foldSize]
testSignal = signal[foldSize:]

x_star = train(trainSignal, 2)
    
        

(799,)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 799 and the array at index 1 has size 0